In [1]:
%pip install whylogs[all] textstat -q


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/home/jamie/projects/v1/whylogs/python/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset imdb (/home/jamie/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 47.66it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
import pandas as pd

df = pd.DataFrame(data=dataset["train"]["text"], columns=["text"])
df.head()

,text
0,I rented I AM CURIOUS-YELLOW from my video sto...
1,"""I Am Curious: Yellow"" is a risible and preten..."
2,If only to avoid making this type of film in t...
3,This film was probably inspired by Godard's Ma...
4,"Oh, brother...after hearing about this ridicul..."


In [5]:
import textstat

# lets look at some scores from textstat,
# nice and numeric so we can just wire these as whylogs UDFs
print(textstat.flesch_kincaid_grade(df["text"][0]))
print(textstat.flesch_kincaid_grade(df["text"][1]))

9.6
10.9


In [6]:
!pip install "/home/jamie/projects/v1/TextMetricsToolkit/dist/whylabs_textmetricstoolkit-0.1.0-py3-none-any.whl" -q


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [7]:
from whylogs.core.resolvers import STANDARD_RESOLVER
from whylogs.core.schema import DeclarativeSchema
from whylogs.experimental.core.metrics.udf_metric import (
    generate_udf_schema,
)

import whylabs_textmetricstoolkit.llm_metrics1

# This is the glue code to wire in any udfs decorated previously
# And this is something Jamie could extend by reading from a metric hub
# but for now we can just decorate a few metrics like the cell above
schema = DeclarativeSchema(
    STANDARD_RESOLVER + generate_udf_schema(),
)

In [8]:
import whylogs as why

# Then loggint the text with the collection of udfs is pretty easy (if slow on 25k rows of text)
results = why.log(df, schema=schema)


In [9]:
# we can upload these to whylabs but lets check if the udfs show up here
results.view().to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,...,udf/smog_index:distribution/q_90,udf/smog_index:distribution/q_95,udf/smog_index:distribution/q_99,udf/smog_index:distribution/stddev,udf/smog_index:types/boolean,udf/smog_index:types/fractional,udf/smog_index:types/integral,udf/smog_index:types/object,udf/smog_index:types/string,udf/smog_index:types/tensor
column,,,,,,,,,,,,,,,,,,,,,
text,24879.188998,24561.455792,25204.84395,0,25000,0,0,NaN,0.0,None,...,12.9,13.8,15.6,2.25992,0,25000,0,0,0,0
